In [1]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@4da81590

In [2]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [3]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@4da81590


org.apache.spark.sql.SparkSession@4da81590

# Data preparation

In [15]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/evgeniy.osipchuk/lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [16]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [17]:
trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

trainDf = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

In [18]:
val Array(trainDfTrain, trainDfTest) = trainDf.randomSplit(Array(0.8, 0.2))

trainDfTrain = [_c0: int, ID: int ... 102 more fields]
trainDfTest = [_c0: int, ID: int ... 102 more fields]


[_c0: int, ID: int ... 102 more fields]

# Train model

In [19]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier, GBTClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [20]:
val vectorAssembler = new VectorAssembler()
    .setInputCols(trainDfTrain.drop("TARGET").columns)
    .setOutputCol("features")

vectorAssembler = vecAssembler_f650e6562b28


vecAssembler_f650e6562b28

In [21]:
val trainDfCleaned = vectorAssembler.transform(trainDfTrain).cache

trainDfCleaned = [_c0: int, ID: int ... 103 more fields]


[_c0: int, ID: int ... 103 more fields]

In [22]:
trainDfCleaned.show(1, 0, true)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 _c0                        | 2                                                                                                                                                                                                                                                                                                                                                                     
 ID                         | 146843                                                                                                                                                                                          

 TRANS_COUNT_ATM_PRC        | 0.810810810810811                                                                                                                                                                                                                                                                                                                                                     
 AMOUNT_RUB_ATM_PRC         | 0.925139591756805                                                                                                                                                                                                                                                                                                                                                     
 TURNOVER_PAYM              | 0.0                                                                                                                                                                                             

 TURNOVER_DYNAMIC_PAYM_1M   | 0.0                                                                                                                                                                                                                                                                                                                                                                   
 CLNT_SALARY_VALUE          | 0.0                                                                                                                                                                                                                                                                                                                                                                   
 TRANS_AMOUNT_TENDENCY3M    | 0.39934204884799707                                                                                                                                                                             

In [33]:
val gbt = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setMaxIter(10)
    .setFeatureSubsetStrategy("auto")
    .fit(trainDfCleaned)

gbt = GBTClassificationModel (uid=gbtc_54457158805e) with 10 trees


lastException: Throwable = null


GBTClassificationModel (uid=gbtc_54457158805e) with 10 trees

In [34]:
val predictions = gbt.transform(
    vectorAssembler.transform(trainDfTest)
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(103,[1,19,21,35,...|[1.09616593590750...|[0.89955879390664...|       0.0|
|     0|(103,[0,1,3,11,21...|[1.29144870543279...|[0.92975274228139...|       0.0|
|     0|(103,[0,1,3,5,9,1...|[0.66085863392775...|[0.78946727325914...|       0.0|
|     0|(103,[0,1,21,35,3...|[1.07964011874232...|[0.89653280117338...|       0.0|
|     0|(103,[0,1,23,29,3...|[0.94863276114795...|[0.86958172358212...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [_c0: int, ID: int ... 106 more fields]


[_c0: int, ID: int ... 106 more fields]

In [35]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0-----------------------------------------------
 probability | [0.8995587939066491,0.10044120609335094] 
only showing top 1 row



In [36]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [37]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [38]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.10044120609335094|
|   0.07024725771860096|
|   0.21053272674085566|
|   0.10346719882661759|
|    0.1304182764178785|
+----------------------+
only showing top 5 rows



In [39]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [40]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_668c31d5fef9
accuracy = 0.8129140811501985


0.8129140811501985

# Predict

In [41]:
val testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("/user/evgeniy.osipchuk/lab05_test.csv")
    .na.fill(0)

testDfData = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [48]:
val realPredictions = gbt.transform(vectorAssembler.transform(testDfData)).cache

realPredictions = [_c0: int, ID: int ... 118 more fields]


[_c0: int, ID: int ... 118 more fields]

In [49]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+-------------------+
|    ID|         prediction|
+------+-------------------+
|519130|0.16884413915558072|
|234045|0.07385903010621486|
|401256|0.07093569005876854|
|551070| 0.0878873640524287|
|367285|0.07269397475310291|
+------+-------------------+
only showing top 5 rows



In [50]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "target")
    .coalesce(1)
    .write
    .option("header", "true")
    .option("sep" ,"\t")
    .mode("overwrite")
    .format("csv")
    .save("lab05.csv")

In [31]:
spark.stop()

# Next steps

- add pipeline functionality `Pipeline().setStages(Array(..))`
- use categories (`StringIndexer`, `OneHotEncoder`)
- fill missing values with better data (not just zeros)
- play with hyper-parameters
- try another model (`LogisticRegression`, `GBTClassifier`, `MultilayerPerceptronClassifier`)
- use cross-validation (`TrainValidationSplit`)
